(1) Packages

In [1]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

  Using cached langchainhub-0.1.21-py3-none-any.whl.metadata (659 bytes)
  Using cached types_requests-2.32.0.20241016-py3-none-any.whl.metadata (1.9 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (252 bytes)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached importlib_resources-6.4.5-py3-none-any.whl.metadata (4.0 kB)
  Using cached kubernetes-31.0.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached mmh3-5.0.1-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached durationpy-0.9-py3-none-any.whl.metadata (338 bytes)
  Using cached coloredlogs-15.0.1-py2.py3

(2) LangSmith -> https://docs.smith.langchain.com/

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama, OllamaEmbeddings

EMBEDDING_MODEL = "nomic-embed-text"
LLM_MODEL = "llama3-groq-tool-use:latest"

# INDEXING

# Load documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
doc_splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(
    documents=doc_splits, embedding=OllamaEmbeddings(model=EMBEDDING_MODEL)
)

retriever = vectorstore.as_retriever()

# RETRIEVAL and GENERATION
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOllama(model=LLM_MODEL, temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke(input="What is Task Decomposition?")

TypeError: RunnableSequence.invoke() missing 1 required positional argument: 'input'